In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import pandas as pd
import numpy as np
import geopandas as gpd
from pydantic import BaseModel
import xarray as xr
import dfm_tools as dfmt
import xugrid as xu
import matplotlib.pyplot as plt
import random

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

import networkx as nx
import ribasim

In [ ]:
%load_ext autoreload
%autoreload 2

Define base directory and results directory

In [ ]:
base_dir = Path("..\\..\\ribasim_lumping_data\\")
results_dir = Path(base_dir, "results")

Define network name and load areas (discharge units: afwaterende eenheden)

In [ ]:
network_name = "zutphen_tki_netwerk"

areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_clip_Zutphen.shp")
areas_gdf = gpd.read_file(areas_file_path)

Create networkanalysis

In [ ]:
network = RibasimLumpingNetwork(name=network_name, areas_gdf=areas_gdf)

Select simulation sets and extract all data points

In [ ]:
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
)
network.add_data_from_simulations_set(
    set_name="zomer",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)

In [ ]:
network.get_network_data()

Define node_ids on which to split the network into Ribasim basins:
- define types to include
- define additional split nodes by id
- define which of the node_ids should be excluded
- combine types of split_node_ids
- add split_nodes to network

In [ ]:
split_node_ids = network.get_node_ids_from_type(
    bifurcations=False,
    confluences=False,
    weirs=True,
    pumps=True,
    laterals=False,
)
split_node_ids_to_include = [1452, 2378, 419, 96]
split_node_ids_to_exclude = [314]

split_node_ids = [node_id for node_id in split_node_ids + split_node_ids_to_include 
                  if node_id not in split_node_ids_to_exclude]

network.add_split_nodes_based_on_node_ids(split_node_ids=split_node_ids);

Create basins (gdf) based on nodes, edges, split_node_ids and areas

In [ ]:
network.create_basins_based_on_split_nodes();

TODO: Find and create ribasim_edges_gdf between basins

In [ ]:
network.basins_gdf.head(3)

In [ ]:
network.split_nodes.head(3)

In [ ]:
network.nodes_gdf.head(3)

In [ ]:
network.edges_gdf.head(3)

In [ ]:

ribasim_edges_gdf = network.split_nodes[['mesh1d_nNodes','geometry']]
ribasim_edges_gdf = ribasim_edges_gdf.rename(columns={"geometry":"geometry_splitnode"})

# merge splitnodes with edges
ribasim_edges_ds = ribasim_edges_gdf.merge(network.edges_gdf[['basin', 'start_node_no', 'end_node_no','mesh1d_nEdges']], left_on='mesh1d_nNodes', right_on='start_node_no')
ribasim_edges_us = ribasim_edges_gdf.merge(network.edges_gdf[['basin', 'start_node_no','end_node_no','mesh1d_nEdges']], left_on='mesh1d_nNodes', right_on='end_node_no')

# DS
# merge splitnodes with basins
ribasim_edges_ds = ribasim_edges_ds.merge(network.basins_gdf[['basin', 'geometry']], left_on='basin', right_on='basin').rename(columns={"geometry":"geometry_basin"})
ribasim_edges_ds['direction'] = 'in'
basin_connections_ds = ribasim_edges_ds.copy()

# draw connection
from shapely.geometry import LineString
ribasim_edges_ds['geometry'] = ribasim_edges_ds.apply(lambda row: LineString([row['geometry_splitnode'], row['geometry_basin']]), axis=1)
ribasim_edges_ds = gpd.GeoDataFrame(ribasim_edges_ds, geometry='geometry', crs=28992)


# US
# merge splitnodes with basins
ribasim_edges_us = ribasim_edges_us.merge(network.basins_gdf[['basin', 'geometry']], left_on='basin', right_on='basin').rename(columns={"geometry":"geometry_basin"})
ribasim_edges_us['direction'] = 'out'
basin_connections_us = ribasim_edges_us.copy()

# draw connection
ribasim_edges_us['geometry'] = ribasim_edges_us.apply(lambda row: LineString([row['geometry_basin'],row['geometry_splitnode']]), axis=1)
ribasim_edges_us = gpd.GeoDataFrame(ribasim_edges_us, geometry='geometry', crs = 28992)

# concat us and ds
ribasim_edges_gdf = pd.concat([ribasim_edges_ds, ribasim_edges_us])
# basin_connections = ribasim_edges_gdf.copy()

ribasim_edges_gdf = ribasim_edges_gdf.drop(columns=['geometry_splitnode','geometry_basin'])

# merge basin connections with nodes
basin_connections_us = basin_connections_us.merge(network.nodes_gdf[['mesh1d_nNodes', 'geometry']], left_on='start_node_no', right_on='mesh1d_nNodes', suffixes=('','_r')).rename(columns={"geometry":"geometry_edge_start_node"})
basin_connections_ds = basin_connections_ds.merge(network.nodes_gdf[['mesh1d_nNodes', 'geometry']], left_on='end_node_no', right_on='mesh1d_nNodes', suffixes=('','_r')).rename(columns={"geometry":"geometry_edge_end_node"})
# basin_connections_us['x'] = basin_connections_us.geometry_edge_start_node.x

basin_connections_us.head(3)

In [ ]:
basin_connections_us['coords'] = basin_connections_us.geometry_basin.apply(lambda p: list(p.coords)[0])
# basin_connections_us['coords'] = (basin_connections_us.geometry_basin.apply(lambda p: p.x) + basin_connections_us.geometry_splitnode.apply(lambda p: p.x))/2
basin_connections_us.head(3)

In [ ]:
# merge upstream and downstream connections
basin_connections_gdf = basin_connections_us.merge(basin_connections_ds, left_on='mesh1d_nNodes',right_on='mesh1d_nNodes',suffixes=('_out','_in'))

# add coordinate in middle of two nodes upstream and downstream of splitpoint
basin_connections_gdf['x'] = (basin_connections_gdf.geometry_edge_start_node.apply(lambda p: p.x) + basin_connections_gdf.geometry_edge_end_node.apply(lambda p: p.x))/2
basin_connections_gdf['y'] = (basin_connections_gdf.geometry_edge_start_node.apply(lambda p: p.y) + basin_connections_gdf.geometry_edge_end_node.apply(lambda p: p.y))/2
basin_connections_gdf['extra_point'] = gpd.points_from_xy(basin_connections_gdf['x'], basin_connections_gdf['y'])

# basin_connections_gdf['geometry'] = basin_connections_gdf.apply(lambda row: LineString([row['geometry_basin_in'],row['geometry_basin_out']]), axis=1)
basin_connections_gdf['geometry'] = basin_connections_gdf.apply(lambda row: LineString([row['geometry_basin_out'],row['extra_point'],row['geometry_basin_in']]), axis=1)
# alternative: draw line via split node
# basin_connections_gdf['geometry'] = basin_connections_gdf.apply(lambda row: LineString([row['geometry_basin_in'],row['geometry_splitnode_out'],row['geometry_basin_out']]), axis=1)
basin_connections_gdf = gpd.GeoDataFrame(basin_connections_gdf, geometry='geometry',crs=28992)

basin_connections_gdf = basin_connections_gdf.drop(columns=['geometry_edge_start_node','geometry_edge_end_node', 'extra_point', 'geometry_splitnode_out','geometry_basin_out','start_node_no_out','mesh1d_nEdges_out','end_node_no_out','direction_out','geometry_splitnode_in','geometry_basin_in','start_node_no_in','end_node_no_in','mesh1d_nEdges_in','direction_in'])

In [ ]:
basin_connections_gdf.head()

In [ ]:
basin_connections_gdf.dtypes

In [ ]:
basin_connections_gdf.plot()
basin_connections_gdf.head(3)

In [ ]:
ribasim_edges_gdf.plot()
ribasim_edges_gdf

In [ ]:
network.ribasim_edges_gdf = ribasim_edges_gdf
network.basin_connections_gdf = basin_connections_gdf

Export everything to geopackage

In [ ]:
network.export_to_geopackage(output_dir=results_dir)